In [35]:
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt


In [36]:
import numpy as np
import datetime as dt
import json

In [37]:
import requests
import pandas as pd
import io
csv_url = "https://data.cdc.gov/api/views/735e-byxc/rows.csv"


In [38]:
# Download the CSV file
response = requests.get(csv_url)
content = response.content

In [39]:
df = pd.read_csv(io.StringIO(content.decode('utf-8')))

In [40]:
selected_columns = ['YearStart', 'LocationDesc','Data_Value','Sample_Size','Stratification1'] 
clean_df = df[selected_columns]
print (clean_df.head(5))


   YearStart   LocationDesc  Data_Value  Sample_Size Stratification1
0       2018     Washington        13.7      12230.0         18 - 23
1       2018         Hawaii        10.2       2192.0        Hispanic
2       2020  Massachusetts        17.6      13520.0        Hispanic
3       2020      Tennessee        18.9       4947.0        Hispanic
4       2012           Guam        12.0       1456.0            Male


In [41]:
clean_df = clean_df.rename(columns={'YearStart': "Year",'LocationDesc':'State','Stratification1':'Category'})



In [42]:
clean_df["Category"].unique()

array(['18 - 23', 'Hispanic', 'Male', 'Non-Hispanic White', '36 - 47',
       'Female', 'Asian/Pacific Islander', 'Non-Hispanic Black',
       '48 - 59', 'American Indian/Alaska Native', '6 - 11', '24 - 35',
       '3 - 5', 'Total', '12 - 17'], dtype=object)

In [43]:
state_year_df = clean_df.groupby(["State", "Year"])["Data_Value"].mean()
print(state_year_df)


State    Year
Alabama  2008    14.591176
         2010    14.858824
         2012    14.723529
         2014    14.991176
         2016    15.261765
                   ...    
Wyoming  2012    12.051515
         2014    10.233333
         2016     9.970588
         2018    10.970968
         2020    11.358065
Name: Data_Value, Length: 378, dtype: float64


In [44]:
state_year_df=state_year_df.reset_index()

In [45]:
state_year_list = []   
i=0 
for index,row in state_year_df.iterrows():
    entry = {}
    entry["State"] = row[0]
    entry["Year"] = row[1]
    entry["Data_Value"] = row[2]
    entry["ID"] = i
    i=i+1
    state_year_list.append(entry)   

In [46]:
state_year_list


[{'State': 'Alabama', 'Year': 2008, 'Data_Value': 14.591176470588236, 'ID': 0},
 {'State': 'Alabama', 'Year': 2010, 'Data_Value': 14.858823529411765, 'ID': 1},
 {'State': 'Alabama', 'Year': 2012, 'Data_Value': 14.723529411764707, 'ID': 2},
 {'State': 'Alabama', 'Year': 2014, 'Data_Value': 14.991176470588234, 'ID': 3},
 {'State': 'Alabama', 'Year': 2016, 'Data_Value': 15.261764705882353, 'ID': 4},
 {'State': 'Alabama', 'Year': 2018, 'Data_Value': 15.09705882352941, 'ID': 5},
 {'State': 'Alabama', 'Year': 2020, 'Data_Value': 14.81764705882353, 'ID': 6},
 {'State': 'Alaska', 'Year': 2008, 'Data_Value': 18.241176470588236, 'ID': 7},
 {'State': 'Alaska', 'Year': 2010, 'Data_Value': 17.908823529411762, 'ID': 8},
 {'State': 'Alaska', 'Year': 2012, 'Data_Value': 18.052941176470586, 'ID': 9},
 {'State': 'Alaska', 'Year': 2014, 'Data_Value': 16.75294117647059, 'ID': 10},
 {'State': 'Alaska', 'Year': 2016, 'Data_Value': 17.358823529411765, 'ID': 11},
 {'State': 'Alaska', 'Year': 2018, 'Data_Value

In [47]:
Category_df = clean_df.sort_values(['Category']).drop_duplicates(subset=['State','Year'], keep='first')     
print (Category_df.head())

      Year          State  Data_Value  Sample_Size Category
6425  2016       New York        10.8      82012.0  12 - 17
5576  2008        Vermont        11.4       1378.0  12 - 17
5619  2012  New Hampshire        13.3       1524.0  12 - 17
5784  2014       Illinois        12.3      27575.0  12 - 17
5812  2012        Indiana        12.4      17041.0  12 - 17


In [48]:
Category_df = clean_df.sort_values(['Category']).drop_duplicates(subset=['Category'], keep='first')
Category_df = Category_df.reset_index(drop=True)
Category_df = Category_df.drop(columns=['Sample_Size'])
Category_df = Category_df.rename(columns={'Category': "ID",'Stratification1':'Category'})
Category_df = Category_df.reset_index(drop=True)
Category_df



,Year,State,Data_Value,ID
0,2016,New York,10.8,12 - 17
1,2018,Louisiana,16.3,18 - 23
2,2008,Vermont,12.5,24 - 35
3,2012,Ohio,9.0,3 - 5
4,2018,Michigan,16.1,36 - 47
5,2020,Illinois,16.7,48 - 59
6,2010,Pennsylvania,12.6,6 - 11
7,2010,Wyoming,18.1,American Indian/Alaska Native
8,2010,Ohio,10.6,Asian/Pacific Islander
9,2020,Ohio,15.2,Female


In [49]:
#Category_df['Category'].plot(kind='bar', figsize=(20,10))

In [50]:
#rouped_df = clean_df.groupby('State').median()
#print (grouped_df.head(20))


In [51]:
#state_year_df.plot(x='Satate', y='Year', rot=90,)
#state_year_df.plot(x='Satate', figsize=(20,10))
#plt.show()

In [52]:
state_year_df.to_csv('state_year_df.csv', index=True, header=True)

In [53]:
json.dumps(state_year_list)


'[{"State": "Alabama", "Year": 2008, "Data_Value": 14.591176470588236, "ID": 0}, {"State": "Alabama", "Year": 2010, "Data_Value": 14.858823529411765, "ID": 1}, {"State": "Alabama", "Year": 2012, "Data_Value": 14.723529411764707, "ID": 2}, {"State": "Alabama", "Year": 2014, "Data_Value": 14.991176470588234, "ID": 3}, {"State": "Alabama", "Year": 2016, "Data_Value": 15.261764705882353, "ID": 4}, {"State": "Alabama", "Year": 2018, "Data_Value": 15.09705882352941, "ID": 5}, {"State": "Alabama", "Year": 2020, "Data_Value": 14.81764705882353, "ID": 6}, {"State": "Alaska", "Year": 2008, "Data_Value": 18.241176470588236, "ID": 7}, {"State": "Alaska", "Year": 2010, "Data_Value": 17.908823529411762, "ID": 8}, {"State": "Alaska", "Year": 2012, "Data_Value": 18.052941176470586, "ID": 9}, {"State": "Alaska", "Year": 2014, "Data_Value": 16.75294117647059, "ID": 10}, {"State": "Alaska", "Year": 2016, "Data_Value": 17.358823529411765, "ID": 11}, {"State": "Alaska", "Year": 2018, "Data_Value": 16.84705

In [54]:
with open('state_year_list.json', 'w') as outfile:
    json.dump(state_year_list, outfile) 

In [55]:
state_year_df.to_json('state_year.json', orient='records')